In [6]:
import pandas as pd
import numpy as np

df = pd.read_excel(r"C:\Users\deena\OneDrive\Desktop\Datasets\Projects to do\marketinganalysisandworkflow\analysis_ready.xlsx")
df.head()


,date,platform,campaign,impressions,clicks,leads,orders,spent,revenue,CTR,Conversion Rate Orders,CPC,ROAS
0,2021-02-01,Facebook,facebook_tier1,148263,1210,13,1,7307.37,4981,0.008161,0.000826,6.039149,0.681641
1,2021-02-01,Facebook,facebook_tier2,220688,1640,48,3,16300.20,14962,0.007431,0.001829,9.939146,0.917903
2,2021-02-01,Google,google_hot,22850,457,9,1,5221.60,7981,0.020000,0.002188,11.425821,1.528459
3,2021-02-01,Google,google_wide,147038,1196,24,1,6037.00,2114,0.008134,0.000836,5.047659,0.350174
4,2021-02-01,Youtube,youtube_blogger,225800,2258,49,10,29962.20,84490,0.010000,0.004429,13.269353,2.819886


In [ ]:
# Marketing Campaign Performance Analysis

## 1. Objective
Evaluate whether marketing campaign performance (ROAS) differs significantly across campaigns and identify which campaigns should be scaled, optimized, or deprioritized.

## 2. Data Overview & Validation

In [7]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 308 entries, 0 to 307
Data columns (total 13 columns):
 #   Column                  Non-Null Count  Dtype         
---  ------                  --------------  -----         
 0   date                    308 non-null    datetime64[ns]
 1   platform                308 non-null    object        
 2   campaign                308 non-null    object        
 3   impressions             308 non-null    int64         
 4   clicks                  308 non-null    int64         
 5   leads                   308 non-null    int64         
 6   orders                  308 non-null    int64         
 7   spent                   308 non-null    float64       
 8   revenue                 308 non-null    int64         
 9   CTR                     308 non-null    float64       
 10  Conversion Rate Orders  308 non-null    float64       
 11  CPC                     308 non-null    float64       
 12  ROAS                    308 non-null    float64   

date                      0
platform                  0
campaign                  0
impressions               0
clicks                    0
leads                     0
orders                    0
spent                     0
revenue                   0
CTR                       0
Conversion Rate Orders    0
CPC                       0
ROAS                      0
dtype: int64

In [ ]:
# The dataset contains 308 observations across 11 campaigns with no missing values and balanced sample sizes (n = 28 per campaign).

In [11]:
df.groupby("campaign")["ROAS"].agg(
    ["mean", "median", "std"]
)

,mean,median,std
campaign,,,
banner_partner,1.242261,1.231464,0.330860
facebook_lal,0.119102,0.112304,0.058626
facebook_retargeting,2.450263,2.114789,1.456362
facebook_tier1,0.933624,0.894421,0.121445
facebook_tier2,0.709152,0.750407,0.220862
google_hot,1.708653,1.805375,0.703801
google_wide,0.674174,0.734403,0.253917
instagram_blogger,1.331378,1.309829,0.186054
instagram_tier1,1.645921,1.771870,0.715182


In [26]:
campaign_summary = (
    df.groupby("campaign")["ROAS"]
      .agg(
          mean_roas="mean",
          median_roas="median",
          std_roas="std",
          n="count"
      )
      .sort_values("mean_roas", ascending=False)
      .reset_index()
)

campaign_summary

,campaign,mean_roas,median_roas,std_roas,n
0,youtube_blogger,4.068700,3.954332,1.014414,28
1,facebook_retargeting,2.450263,2.114789,1.456362,28
2,google_hot,1.708653,1.805375,0.703801,28
3,instagram_tier1,1.645921,1.771870,0.715182,28
4,instagram_blogger,1.331378,1.309829,0.186054,28
5,banner_partner,1.242261,1.231464,0.330860,28
6,facebook_tier1,0.933624,0.894421,0.121445,28
7,facebook_tier2,0.709152,0.750407,0.220862,28
8,google_wide,0.674174,0.734403,0.253917,28
9,instagram_tier2,0.600499,0.599109,0.274449,28


In [ ]:
# Initial descriptive statistics suggest substantial performance differences across campaigns, with YouTube Blogger and Facebook Retargeting exhibiting the highest ROAS.

In [13]:
from scipy.stats import shapiro

normality = df.groupby("campaign")["ROAS"].apply(
    lambda x: shapiro(x)[1]
)

normality

campaign
banner_partner          0.000691
facebook_lal            0.528170
facebook_retargeting    0.112996
facebook_tier1          0.148102
facebook_tier2          0.000003
google_hot              0.000548
google_wide             0.009345
instagram_blogger       0.227767
instagram_tier1         0.001804
instagram_tier2         0.016494
youtube_blogger         0.138234
Name: ROAS, dtype: float64

In [15]:
from scipy.stats import levene

groups = [
    g["ROAS"].values
    for _, g in df.groupby("campaign")
]

levene_p = levene(*groups).pvalue
levene_p

2.690024936002042e-21

In [ ]:
## 3. Assumption Testing

### Normality
Shapiro–Wilk tests indicate that ROAS is not normally distributed for several campaigns, which is common in marketing performance data.

### Homogeneity of Variance
Levene’s test strongly rejects equal variances across campaigns (p < 0.001), indicating heteroskedasticity (The spread of errors is not consistent as some parts of the data have more “noise” than others)

In [ ]:
## 4. Hypothesis Testing (Welch ANOVA)

# Due to non-normality and unequal variances, Welch’s ANOVA was selected as a robust alternative to classic ANOVA.

H₀: Mean ROAS is equal across campaigns  
H₁: At least one campaign differs in mean ROAS


In [16]:
from statsmodels.stats.oneway import anova_oneway

welch = anova_oneway(
    df["ROAS"],
    groups=df["campaign"],
    use_var="unequal"
)

welch


<class 'statsmodels.stats.base.HolderTuple'>
statistic = 262.5855412021496
pvalue = 6.089757893723092e-74
df = (10.0, 114.32799949268303)
df_num = 10.0
df_denom = 114.32799949268303
nobs_t = 308.0
n_groups = 11
means = array([1.24226054, 0.11910185, 2.45026263, 0.9336241 , 0.70915233,
           1.70865333, 0.67417411, 1.3313781 , 1.64592078, 0.60049887,
           4.06870044])
nobs = array([28., 28., 28., 28., 28., 28., 28., 28., 28., 28., 28.])
vars_ = array([0.10946862, 0.00343704, 2.12098999, 0.0147488 , 0.04877992,
           0.49533611, 0.06447392, 0.03461594, 0.51148529, 0.07532209,
           1.02903496])
use_var = 'unequal'
welch_correction = True
tuple = (262.5855412021496, 6.089757893723092e-74)

In [ ]:
# The Welch ANOVA result (p < 0.001) indicates statistically significant differences in ROAS across campaigns.

In [17]:
ss_between = welch.statistic * (len(df) - 1)
ss_total = np.sum((df["ROAS"] - df["ROAS"].mean())**2)
eta_sq = ss_between / ss_total
eta_sq

177.08682553643663

In [ ]:
# The calculated eta squared (η²) indicates a large effect size, meaning campaign selection explains a substantial proportion of ROAS variation. This suggests the observed differences are not only statistically significant but also practically meaningful for business decisions.

In [19]:
from statsmodels.stats.multicomp import pairwise_tukeyhsd

tukey = pairwise_tukeyhsd(
    df["ROAS"],
    df["campaign"],
    alpha=0.05
)

tukey.summary()

group1,group2,meandiff,p-adj,lower,upper,reject
banner_partner,facebook_lal,-1.1232,0.0,-1.6782,-0.5681,True
banner_partner,facebook_retargeting,1.208,0.0,0.653,1.763,True
banner_partner,facebook_tier1,-0.3086,0.7771,-0.8636,0.2464,False
banner_partner,facebook_tier2,-0.5331,0.0723,-1.0881,0.0219,False
banner_partner,google_hot,0.4664,0.1934,-0.0886,1.0214,False
banner_partner,google_wide,-0.5681,0.0397,-1.1231,-0.0131,True
banner_partner,instagram_blogger,0.0891,1.0,-0.4659,0.6441,False
banner_partner,instagram_tier1,0.4037,0.3967,-0.1514,0.9587,False
banner_partner,instagram_tier2,-0.6418,0.0095,-1.1968,-0.0868,True
banner_partner,youtube_blogger,2.8264,0.0,2.2714,3.3815,True


In [27]:
tukey_df = pd.DataFrame(
    tukey._results_table.data[1:],
    columns=tukey._results_table.data[0]
)

tukey_df["significant"] = tukey_df["reject"].astype(bool)
tukey_df

,group1,group2,meandiff,p-adj,lower,upper,reject,significant
0,banner_partner,facebook_lal,-1.1232,0.0000,-1.6782,-0.5681,True,True
1,banner_partner,facebook_retargeting,1.2080,0.0000,0.6530,1.7630,True,True
2,banner_partner,facebook_tier1,-0.3086,0.7771,-0.8636,0.2464,False,False
3,banner_partner,facebook_tier2,-0.5331,0.0723,-1.0881,0.0219,False,False
4,banner_partner,google_hot,0.4664,0.1934,-0.0886,1.0214,False,False
5,banner_partner,google_wide,-0.5681,0.0397,-1.1231,-0.0131,True,True
6,banner_partner,instagram_blogger,0.0891,1.0000,-0.4659,0.6441,False,False
7,banner_partner,instagram_tier1,0.4037,0.3967,-0.1514,0.9587,False,False
8,banner_partner,instagram_tier2,-0.6418,0.0095,-1.1968,-0.0868,True,True
9,banner_partner,youtube_blogger,2.8264,0.0000,2.2714,3.3815,True,True


In [28]:
tier_map = {
    "youtube_blogger": "Tier 1 – Scale",
    "facebook_retargeting": "Tier 2 – Optimize",
    "google_hot": "Tier 2 – Optimize",
    "instagram_tier1": "Tier 2 – Optimize",
    "banner_partner": "Tier 3 – Monitor",
    "instagram_blogger": "Tier 3 – Monitor",
    "facebook_tier1": "Tier 3 – Monitor",
    "facebook_lal": "Tier 4 – Deprioritize",
    "instagram_tier2": "Tier 4 – Deprioritize",
    "google_wide": "Tier 4 – Deprioritize",
    "facebook_tier2": "Tier 4 – Deprioritize"
}

campaign_summary["tier"] = campaign_summary["campaign"].map(tier_map)
campaign_summary


,campaign,mean_roas,median_roas,std_roas,n,tier
0,youtube_blogger,4.068700,3.954332,1.014414,28,Tier 1 – Scale
1,facebook_retargeting,2.450263,2.114789,1.456362,28,Tier 2 – Optimize
2,google_hot,1.708653,1.805375,0.703801,28,Tier 2 – Optimize
3,instagram_tier1,1.645921,1.771870,0.715182,28,Tier 2 – Optimize
4,instagram_blogger,1.331378,1.309829,0.186054,28,Tier 3 – Monitor
5,banner_partner,1.242261,1.231464,0.330860,28,Tier 3 – Monitor
6,facebook_tier1,0.933624,0.894421,0.121445,28,Tier 3 – Monitor
7,facebook_tier2,0.709152,0.750407,0.220862,28,Tier 4 – Deprioritize
8,google_wide,0.674174,0.734403,0.253917,28,Tier 4 – Deprioritize
9,instagram_tier2,0.600499,0.599109,0.274449,28,Tier 4 – Deprioritize


In [ ]:
# Post-hoc testing was used to group campaigns into statistically defensible performance tiers to support budget allocation decisions.

In [32]:
!pip install xlsxwriter

In [35]:
import os
import xlsxwriter

# Input file
input_path = r"C:\Users\deena\OneDrive\Desktop\Datasets\Projects to do\marketinganalysisandworkflow\analysis_ready.xlsx"

# Load data
df = pd.read_excel(input_path)

# Extract folder of input file
input_folder = os.path.dirname(input_path)

# Build output path in same folder
output_path = os.path.join(input_folder, "statistical_results.xlsx")

# Create results DataFrame
anova_results = pd.DataFrame({
    "test": ["Welch ANOVA"],
    "f_statistic": [welch.statistic],
    "p_value": [welch.pvalue],
    "eta_squared": [eta_sq],
    "interpretation": ["Large effect"]
})

# Export to Excel
with pd.ExcelWriter(output_path, engine="xlsxwriter") as writer:
    campaign_summary.to_excel(writer, sheet_name="campaign_summary", index=False)
    tukey_df.to_excel(writer, sheet_name="tukey_results", index=False)
    anova_results.to_excel(writer, sheet_name="anova_results", index=False)


In [ ]:
## 6. Outputs

Statistical results have been exported to Excel for dashboarding and stakeholder communication. All statistical inference was performed in Python to ensure methodological rigor and reproducibility.